In [1]:
import numpy as np

import zarr_io
import data_prep
import coarse_registration

from sofima import stitch_rigid

In [2]:
# Preprocess raw tiles
bucket = 'sofima-test-bucket'
downsampling_exp = 2
path_0 = f'tile_X_0001_Y_0000_Z_0000_CH_0405_cam0.zarr/{downsampling_exp}'  
path_1 = f'tile_X_0002_Y_0000_Z_0000_CH_0405_cam0.zarr/{downsampling_exp}'
tile_0 = zarr_io.open_zarr(bucket, path_0)
tile_1 = zarr_io.open_zarr(bucket, path_1)
data_prep.preprocess_tiles(tile_volumes = [tile_0, tile_1], 
                           output_names = ['preprocessed_0.zarr', 'preprocessed_1.zarr'],
                           bucket = bucket, 
                           batch_size = 256)

I0000 00:00:1684359504.139048  191820 gcs_resource.cc:102] Using default AdmissionQueue with limit 32
I0000 00:00:1684359504.140731  191849 google_auth_provider.cc:179] Running on GCE, using service account 895865026362-compute@developer.gserviceaccount.com


CPU Only: 8.5 minutes

GPU, Batch Size = 32 = 2**5: 4.5 minutes

GPU, Batch Size = 256 = 2**8: 1.6 minutes

GPU, Batch Size = 512 = 2**9: 1.5 minutes

GPU, Batch Size = 1024 = 2**10: 1.5 minutes

In [2]:
# Load preprocessed tiles:  
bucket = 'sofima-test-bucket'
path_0 = f'preprocessed_0.zarr'
path_1 = f'preprocessed_1.zarr'
tile_0 = zarr_io.open_zarr(bucket, path_0)
tile_1 = zarr_io.open_zarr(bucket, path_1)

I0000 00:00:1684359760.771328  194652 gcs_resource.cc:102] Using default AdmissionQueue with limit 32
I0000 00:00:1684359760.773134  194937 google_auth_provider.cc:179] Running on GCE, using service account 895865026362-compute@developer.gserviceaccount.com


In [3]:
# Coarse Registration
tile_layout = np.array([[0], 
                        [1]])
tile_map = {(0, 0):0, (1, 0):1}  # (ty, tx) -> id
idx_to_coord = {0:(0, 0), 1:(1, 0)}
tile_volumes = [tile_0.T[:,:,:,0,0], tile_1.T[:,:,:,0,0]]  # Place X/Y dimensions in front, must have 3D
cx, cy = coarse_registration.compute_coarse_offsets(tuple(tile_layout.shape), tile_map, tile_volumes)
coarse_mesh = stitch_rigid.optimize_coarse_mesh(cx, cy, mesh_fn=stitch_rigid.elastic_tile_mesh_3d)

print(f'{cx=}')
print(f'{cy=}')

(100, 300, 100)
(50, 50, 50)
[ 25 125  25]
[  0 401   0]
[  -5. -114.    6.]
cx=array([[[[nan],
         [nan]]],


       [[[nan],
         [nan]]],


       [[[nan],
         [nan]]]])
cy=array([[[[ -5.],
         [ nan]]],


       [[[287.],
         [ nan]]],


       [[[  6.],
         [ nan]]]])


uint16 images: 90s

uint8 images: 20s

In [4]:
# Finally, save result for fine registration
np.savez_compressed('coarse_results.npz', mesh=coarse_mesh, cx=cx, cy=cy)

Current set of parameters: 
- QUERY_R_ORTHO = 25
- QUERY_R_OVERLAP = 25
- SEARCH_OVERLAP = 300
- SEARCH_R_ORTHO = 50

Result, zyx: [-5, 287, 6] 
Heurisitc: Small search area and small search patch works best.  